# Example of transformer with non-trivial phase shift and tap ratio

This example is a copy of pandapower's minimal example.

In [1]:
import pypsa
import numpy as np

In [2]:
network = pypsa.Network()

network.add("Bus","MV bus",v_nom=20,v_mag_pu_set=1.02)
network.add("Bus","LV1 bus",v_nom=.4)
network.add("Bus","LV2 bus",v_nom=.4)

network.add("Transformer","MV-LV trafo",type="0.4 MVA 20/0.4 kV",bus0="MV bus",bus1="LV1 bus")

network.add("Line","LV cable",type="NAYY 4x50 SE",bus0="LV1 bus",bus1="LV2 bus",length=0.1)

network.add("Generator","External Grid",bus="MV bus",control="Slack")

network.add("Load","LV load",bus="LV2 bus",p_set=0.1,q_set=0.05)

In [3]:
def run_pf():
    network.lpf()
    network.pf(use_seed=True)
    print("Voltage angles:")
    print(network.buses_t.v_ang*180./np.pi)
    print("Voltage magnitudes:")
    print(network.buses_t.v_mag_pu)

In [4]:
run_pf()

INFO:numexpr.utils:NumExpr defaulting to 8 threads.
INFO:pypsa.pf:Performing linear load-flow on AC sub-network SubNetwork 0 for snapshot(s) Index(['now'], dtype='object')
INFO:pypsa.pf:Performing non-linear load-flow on AC sub-network SubNetwork 0 for snapshots Index(['now'], dtype='object')
INFO:pypsa.pf:Newton-Raphson solved in 3 iterations with error of 0.000000 in 0.025899 seconds


Voltage angles:
     MV bus     LV1 bus     LV2 bus
now     0.0 -150.760126 -149.884141
Voltage magnitudes:
     MV bus   LV1 bus   LV2 bus
now    1.02  1.008843  0.964431


In [5]:
network.transformers.tap_position = 2
run_pf()

INFO:pypsa.pf:Performing linear load-flow on AC sub-network SubNetwork 0 for snapshot(s) Index(['now'], dtype='object')
INFO:pypsa.pf:Performing non-linear load-flow on AC sub-network SubNetwork 0 for snapshots Index(['now'], dtype='object')
INFO:pypsa.pf:Newton-Raphson solved in 3 iterations with error of 0.000000 in 0.024260 seconds


Voltage angles:
     MV bus     LV1 bus     LV2 bus
now     0.0 -150.843911 -149.870837
Voltage magnitudes:
     MV bus   LV1 bus   LV2 bus
now    1.02  0.959655  0.912713


In [6]:
network.transformers.tap_position = -2
run_pf()

INFO:pypsa.pf:Performing linear load-flow on AC sub-network SubNetwork 0 for snapshot(s) Index(['now'], dtype='object')
INFO:pypsa.pf:Performing non-linear load-flow on AC sub-network SubNetwork 0 for snapshots Index(['now'], dtype='object')
INFO:pypsa.pf:Newton-Raphson solved in 3 iterations with error of 0.000000 in 0.021799 seconds


Voltage angles:
     MV bus     LV1 bus     LV2 bus
now     0.0 -150.681666 -149.896634
Voltage magnitudes:
     MV bus   LV1 bus   LV2 bus
now    1.02  1.063133  1.021202


## Now play with tap changer on LV side

In [7]:
new_trafo_lv_tap = network.transformer_types.loc[["0.4 MVA 20/0.4 kV"]]
new_trafo_lv_tap.index = ["New trafo"]
new_trafo_lv_tap.tap_side = 1
new_trafo_lv_tap

attribute,f_nom,s_nom,v_nom_0,v_nom_1,vsc,vscr,pfe,i0,phase_shift,tap_side,tap_neutral,tap_min,tap_max,tap_step,references
New trafo,50.0,0.4,20.0,0.4,6.0,1.425,1.35,0.3375,150.0,1,0,-2,2,2.5,pandapower;Oswald - Transformatoren - Vorlesun...


In [8]:
network.transformer_types = network.transformer_types.append(new_trafo_lv_tap)
network.transformers.type = "New trafo"

In [9]:
network.transformers.tap_position = 2
run_pf()

INFO:pypsa.pf:Slack bus for sub-network 0 is MV bus
INFO:pypsa.pf:Performing linear load-flow on AC sub-network SubNetwork 0 for snapshot(s) Index([u'now'], dtype='object')
INFO:pypsa.pf:Slack bus for sub-network 0 is MV bus
INFO:pypsa.pf:Performing non-linear load-flow on AC sub-network SubNetwork 0 for snapshots Index([u'now'], dtype='object')
INFO:pypsa.pf:Newton-Raphson solved in 3 iterations with error of 0.000000 in 0.070072 seconds


Voltage angles:
     MV bus    LV1 bus     LV2 bus
now     0.0 -150.75582 -149.964875
Voltage magnitudes:
     MV bus   LV1 bus   LV2 bus
now    1.02  1.059317  1.017221


In [10]:
network.transformers.tap_position = -2
run_pf()

INFO:pypsa.pf:Slack bus for sub-network 0 is MV bus
INFO:pypsa.pf:Performing linear load-flow on AC sub-network SubNetwork 0 for snapshot(s) Index([u'now'], dtype='object')
INFO:pypsa.pf:Slack bus for sub-network 0 is MV bus
INFO:pypsa.pf:Performing non-linear load-flow on AC sub-network SubNetwork 0 for snapshots Index([u'now'], dtype='object')
INFO:pypsa.pf:Newton-Raphson solved in 3 iterations with error of 0.000000 in 0.036843 seconds


Voltage angles:
     MV bus     LV1 bus     LV2 bus
now     0.0 -150.765232 -149.789394
Voltage magnitudes:
     MV bus   LV1 bus   LV2 bus
now    1.02  0.958366  0.911353


## Now make sure that the phase shift is also there in the LOPF

In [11]:
network.generators.p_nom = 1.
network.lines.s_nom = 1.
network.lopf()

INFO:pypsa.pf:Slack bus for sub-network 0 is MV bus
INFO:pypsa.opf:Performed preliminary steps
INFO:pypsa.opf:Building pyomo model using `angles` formulation
INFO:pypsa.opf:Solving model using glpk
INFO:pypsa.opf:Optimization successful


('ok', 'optimal')

In [12]:
print("Voltage angles:")
print(network.buses_t.v_ang*180./np.pi)
print("Voltage magnitudes:")
print(network.buses_t.v_mag_pu)

Voltage angles:
     MV bus     LV1 bus     LV2 bus
now     0.0 -150.793123 -151.090345
Voltage magnitudes:
     MV bus  LV1 bus  LV2 bus
now     1.0      1.0      1.0
